In [1]:
import pickle
import json
import random
import pandas as pd
import numpy as np
import pathlib
import unicodedata

#### Looking at L1 production gold citations

In [150]:
direct_gold = pd.read_parquet("<direct-gold-citations>")

In [165]:
# DF of gold citations
gold_cites = direct_gold.drop_duplicates(subset=['gold_citation'])[['gold_citation']].copy()
gold_cites['ind_col'] = 1
gold_cites.shape

(124577, 2)

In [210]:
def save_pickle(dictionary, file_path):
    # Save the dictionary as a pickle file
    with open(file_path, 'wb') as f:
        pickle.dump(dictionary, f)

In [162]:
def return_empty_list(some_list):
    if isinstance(some_list, list):
        return some_list
    else:
        return []

In [156]:
# DF of works that cite a gold citation
prod_cites = pd.read_parquet("<file-for-gold-citations>", columns=['paper_id','new_L1_links'])
prod_cites.shape

(39696933, 2)

In [157]:
prod_cites['new_L1_links'] = prod_cites['new_L1_links'].apply(lambda x: x.tolist())

In [163]:
gold_merged = gold_cites.rename(columns={'gold_citation':'paper_id'})\
    .merge(prod_cites, how='left', on='paper_id')[['paper_id','new_L1_links']].copy()

gold_merged['new_L1_links'] = gold_merged['new_L1_links'].apply(return_empty_list)

In [172]:
non_gold_merged = prod_cites.merge(gold_cites.rename(columns={'gold_citation':'paper_id'}), 
                                   how='left', on='paper_id')
non_gold_merged = non_gold_merged[non_gold_merged['ind_col']!=1.0][['paper_id','new_L1_links']].copy()
non_gold_merged.shape

(39619175, 2)

In [173]:
gold_dict = gold_merged.set_index('paper_id')['new_L1_links'].to_dict()

In [174]:
non_gold_dict = non_gold_merged.set_index('paper_id')['new_L1_links'].to_dict()

In [211]:
_ = save_pickle(gold_dict, './full_model_iter6/model_artifacts/gold_citations_dict.pkl')

In [212]:
_ = save_pickle(non_gold_dict, './full_model_iter6/model_artifacts/non_gold_citations_dict.pkl')

In [197]:
test_cites_text = """https://openalex.org/W182557598",
"https://openalex.org/W198469451",
"https://openalex.org/W1480706413",
"https://openalex.org/W1492376954",
"https://openalex.org/W1495672122",
"https://openalex.org/W1497663772",
"https://openalex.org/W1498187417",
"https://openalex.org/W1517051009",
"https://openalex.org/W1520425169",
"https://openalex.org/W1533713644",
"https://openalex.org/W1538419182",
"https://openalex.org/W1564805059",
"https://openalex.org/W1573363574",
"https://openalex.org/W1574254013",
"https://openalex.org/W1594827217",
"https://openalex.org/W1959848155",
"https://openalex.org/W1963844966",
"https://openalex.org/W1964081206",
"https://openalex.org/W1964825240",
"https://openalex.org/W1965038309",
"https://openalex.org/W4230026896",
"https://openalex.org/W4243903519"""

test_cites = [int(x.replace("""\"""", "").replace(',','').replace('\n','')) for x in 
              test_cites_text.split("https://openalex.org/W") if x !='']

In [200]:
len(test_cites)

239

In [209]:
%%time
level_0, level_1 = get_gold_citations_from_all_citations(test_cites, gold_dict, non_gold_dict)

CPU times: user 555 µs, sys: 100 µs, total: 655 µs
Wall time: 669 µs
